In [1]:
import math

import osmnx as ox
import networkx as nx

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt  

from shapely.geometry import box
from geopy.distance import distance

In [ ]:
df=gpd.read_file(r"../data/yerevan_boundary/yerevan.shp")

In [4]:
df.set_crs(epsg=4326, inplace=True)

,FID,geometry
0,0,"POLYGON ((44.41298 40.18969, 44.41155 40.19399..."
1,1,"POLYGON ((44.41772 40.18316, 44.41418 40.18603..."
2,2,"POLYGON ((44.42035 40.18107, 44.42025 40.18110..."
3,3,"POLYGON ((44.42598 40.17664, 44.42576 40.17920..."
4,4,"POLYGON ((44.42652 40.17009, 44.42598 40.17664..."
...,...,...
544,544,"POLYGON ((44.57644 40.20351, 44.58512 40.20354..."
545,545,"POLYGON ((44.57648 40.19696, 44.58516 40.19699..."
546,546,"POLYGON ((44.57652 40.19040, 44.58520 40.19043..."
547,547,"POLYGON ((44.57656 40.18385, 44.58524 40.18388..."


In [7]:
df['dummy'] = 1

# Dissolve all polygons into a single polygon
df = df.dissolve(by='dummy')

In [8]:
df

,geometry,FID
dummy,,
1,"MULTIPOLYGON (((44.44686 40.12255, 44.44415 40...",0


In [9]:
# Step 1: Project to a CRS in meters
df_meters = df.to_crs(epsg=3857)  # Web Mercator (meters)

# Step 2: Apply a 500-meter buffer
df_buffered = df_meters.buffer(1500)

# Step 3: Optional - Convert back to original CRS (4326)
df = gpd.GeoDataFrame(geometry=df_buffered, crs=df_meters.crs).to_crs(epsg=4326)

In [10]:
df.geometry.area

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13532\4209366313.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df.geometry.area


dummy
1    0.037305
dtype: float64

In [12]:
def make_grid(gdf, cell_width_m, cell_height_m):
    bounds = gdf.total_bounds
    min_x, min_y, max_x, max_y = bounds
    width = max_x - min_x
    height = max_y - min_y
    
    # Get the latitude at the center of the bounding box
    center_lat = (min_y + max_y) / 2
    
    # Convert cell width and height from meters to degrees
    cell_width = cell_width_m / distance((center_lat, min_x), (center_lat, min_x + 1)).meters
    cell_height = cell_height_m / distance((min_y, center_lat), (min_y + 1, center_lat)).meters

    rows = int(math.ceil(height / cell_height))
    cols = int(math.ceil(width / cell_width))

    grid_cells = []
    for i in range(cols):
        for j in range(rows):
            x1 = min_x + i * cell_width
            y1 = min_y + j * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height
            grid_cells.append(box(x1, y1, x2, y2))
    
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=gdf.crs)
    return grid

# Define the cell width and height in meters
cell_width_m = 545  # Corrected width
cell_height_m = 305  # Height remains the same

# Create the grid
grid = make_grid(df, cell_width_m, cell_height_m)

# Clip the grid to the city boundary
grid_clipped = gpd.clip(grid, df)

In [13]:
grid_clipped["Index"]=range(0,len(grid_clipped))

In [15]:
grid_clipped

,geometry,Index
2084,"POLYGON ((44.56442 40.06911, 44.56528 40.06911...",0
2085,"POLYGON ((44.56442 40.06911, 44.56442 40.07185...",1
2086,"POLYGON ((44.56442 40.07185, 44.56442 40.07460...",2
2087,"POLYGON ((44.56442 40.07460, 44.56442 40.07735...",3
2088,"POLYGON ((44.56442 40.07735, 44.56442 40.08009...",4
...,...,...
2238,"POLYGON ((44.57081 40.26961, 44.57081 40.27026...",2250
1918,"POLYGON ((44.55162 40.27236, 44.55162 40.26961...",2251
1999,"POLYGON ((44.55162 40.27236, 44.55162 40.27267...",2252
1919,"POLYGON ((44.55162 40.27236, 44.54889 40.27236...",2253


In [ ]:
grid_clipped.to_file("../data/yerevan_grid.geojson",driver="GeoJSON")